### M&uuml;hle-Widget, Skizze  

Um das Spielfeld darzustellen, gehen wir von einem M&uuml;hle-Ring mit Zentrum `(0,0)` und Seitenl&auml;nge 2 aus, mit den Punkten
(im Gegenuhrzeigersinn):  
`pts =  [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]`  

Mit Hilfe dieser Punkte rechnen wir eine M&uuml;hle-Koordinate `(r,i)` in eine xy-Koordinate um:
- Sei `x, y = pts[i]` der ite Punkt auf dem dem Ring um `(0,0)`.
- Je nach Ring, strecke/multipliziere `x` mit `1/3 * width/2`,  `2/3 * width/2` oder `3/3* width/2`.  
  Um die Ringmitte in die Leinwadnmitte zu bringen, addiere noch `width/2`.
  Mir der `y`-Koordinate verfahren wir entsprechend.
 
```python
def ri2xy(r, i):
    x, y = pts[i]
    x = ((r+1)/3*x + 1) * width/2
    y = ((r+1)/3*y + 1) * height/2
    return x, y 
```

Eine Variable `ri` speichere ein Tuple `(r, i)`, eine Variable `ris` (Plural) eine Liste mit Tupeln der Form `(r, i)`.

In [ ]:
from muehle0 import Game
from ipycanvas import MultiCanvas

class Widget:
    
    line_width = 2
    scale = 0.8
    pts =  [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
    
    def __init__(self, game, width=200, height=200, color_0='red', color_1='blue'):
        self.colors = {0: color_0, 1: color_1}
        self.radius = min(width, height) / 30
        
        self.mcanvas = MultiCanvas(2, width=width, height=height, 
                layout = {'border' : '1px solid black'},
               )
        self.bg, self.fg = self.mcanvas
        self.bg.line_width = self.line_width
        
        self.game = game
        self.game.callback = self.update
        self.game.new_game()
        
    def ri2xy(self, ri):
        r, i = ri
        s, w, h = self.scale, self.mcanvas.width, self.mcanvas.height
        x, y = self.pts[i]
        x = (s/3*(r+1)*x + 1) * w/2
        y = (s/3*(r+1)*y + 1) * h/2
       
        return x, y    
        
    def new_game(self, muehlen):
        self.mcanvas.clear()
        for muehle in muehlen:
            line = [self.ri2xy(ri) for ri in muehle]
            self.bg.stroke_lines(line)
     
    def place_stone(self, ri, player):
        color = self.colors[player]
        x, y = self.ri2xy(ri)
        self.fg.fill_style = color
        self.fg.fill_circle(x, y, self.radius)
    
    def remove_stone(self, ri):
        x, y = self.ri2xy(ri)
        x -= (self.radius + 1)
        y -= (self.radius + 1)
        self.fg.clear_rect(x, y, 2*self.radius + 2)
        
    def move_stone(self, ri1, ri2, player):
        self.remove_stone(ri1)
        self.place_stone(ri2, player)
        
    def update(self, event, data):
        getattr(self, event)(*data)
    
    def _ipython_display_(self):
        display(self.mcanvas)

In [ ]:
game = Game()
Widget(game, width=300)

### Testen, ob die Game-Instanz mit dem Widget kommuniziert:

In [ ]:
# Muehle bilden
for pos in [(0, 0), (0, 1), (0, 2)]:
    game.move(0, 'p', pos)

In [ ]:
# Stein bewegen
game.move(0, 'm', (0, 0), (0, 7))

In [ ]:
game.move(0, 'm', (0, 7), (0, 0))

In [ ]:
# Stein von Spieler 1 setzen
game.move(1, 'p', (2, 0))

In [ ]:
# Stein wegnehmen
game.move(0, 'r', (2, 0))

### Maus und Tastaturanbindung  
(vgl. Notebooks im Ordner `Stein_auf_Punkt_setzen` der Lektion 15).  
In der Methode `__init__(self)` registrieren wir die Callbacks und stellen eine Liste `xys` mit den Punkten, die wir mit der Maus greifen wollen bereit. In `self.selected_idx` speichern wir den Index eines angeklickten Punktes.  

Dr&uuml;cken vom 'm' bez. 'p' wechselt in die *place* bez. *move*-Phase.

```python
    self.mcanvas.on_mouse_down(self.on_mouse_down)
    self.mcanvas.on_mouse_up(self.on_mouse_up)
    self.mcanvas.on_key_down(self.on_key_down)

    self.ris = [(r,i) for r in range(3) for i in range(8)]
    self.xys = [self.ri2xy(ri) for ri in self.ris]
    self.selected_idx = None
```



In [ ]:
from muehle0 import Game
from get_closest import get_closest
from ipycanvas import MultiCanvas

# zum entwickeln/debuggen
from ipywidgets import Output
debug = Output(layout = {'border': '1px solid black'})

class View:
    
    line_width = 2
    scale = 0.8
    pts =  [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
    
    def __init__(self, game, width=200, height=200, color_0='red', color_1='blue'):
        self.colors = {0: color_0, 1: color_1}
        self.radius = min(width, height) / 30
        
        self.mcanvas = MultiCanvas(2, width = width, height = height, 
                layout = {'border' : '1px solid black'},
               )
        self.bg, self.fg = self.mcanvas
        self.bg.line_width = self.line_width
        
        self.mcanvas.on_mouse_down(self.on_mouse_down)
        self.mcanvas.on_mouse_up(self.on_mouse_up)
        self.mcanvas.on_key_down(self.on_key_down)
       
        self.ris = [(r,i) for r in range(3) for i in range(8)]
        self.xys = [self.ri2xy(ri) for ri in self.ris]
        self.selected_idx = None
      
        self.game = game
        self.game.callback = self.update
        self.game.new_game()
        
    def ri2xy(self, ri):
        r, i = ri
        s, w, h = self.scale, self.mcanvas.width, self.mcanvas.height
        x, y = self.pts[i]
        x = (s/3*(r+1)*x + 1) * w/2
        y = (s/3*(r+1)*y + 1) * h/2
       
        return x, y    
    
    @debug.capture()        
    def on_mouse_down(self, x, y):
        if (idx := get_closest(self.xys, (x, y))) is None:
            return
        
        src = self.ris[idx]
        ptm = self.game.ptm
        
        if self.game.muehle:
            print(ptm, 'remove', src)
            self.game.move(ptm, 'r', src)
        elif self.game.phase == 'place':
            print(ptm, 'place', src)
            self.game.move(ptm, 'p', src)
        else:
            self.selected_idx = idx
                
    @debug.capture()   
    def on_mouse_up(self, x, y):   
        if self.selected_idx is None or self.game.phase != 'move':
            return
        
        idx = get_closest(self.xys, (x, y))
        if idx is not None:
            target = self.ris[idx]
            src = self.ris[self.selected_idx]
            ptm = self.game.ptm
            self.game.move(ptm, 'm', src, target)
            print(ptm, src, target)
           
        self.selected_idx = None 
        
    @debug.capture()   
    def on_key_down(self, key, *flags):   
        if key == 'n':
            self.game.new_game()
        if key == 'p':
            self.game.phase = 'place'
            print('switched phase to place')
        elif key == 'm':
            self.game.phase = 'move'
            print('switched phase to move')
            
    # Methoden fuer Callback
    def new_game(self, muehlen):
        debug.clear_output()
        self.mcanvas.clear()
        for muehle in muehlen:
            line = [self.ri2xy(ri) for ri in muehle]
            self.bg.stroke_lines(line)
     
    def place_stone(self, ri, player):
        color = self.colors[player]
        x, y = self.ri2xy(ri)
        self.fg.fill_style = color
        self.fg.fill_circle(x, y, self.radius)
    
    def remove_stone(self, ri):
        x, y = self.ri2xy(ri)
        x -= (self.radius + 1)
        y -= (self.radius + 1)
        self.fg.clear_rect(x, y, 2*self.radius + 2)
        
    def move_stone(self, ri1, ri2, player):
        self.remove_stone(ri1)
        self.place_stone(ri2, player)
        
    def update(self, event, data):
        getattr(self, event)(*data)
    
    def _ipython_display_(self):
        display(self.mcanvas, debug)
        self.mcanvas.focus()

In [ ]:
game = Game()
view = View(game, width=300)
view